In [20]:
# Fase de encodeo!

import numpy as np
import pandas as pd
import pickle
from scipy.sparse import load_npz, csr_matrix, save_npz
from datetime import datetime

def cargar_pkls():
    try:
        with open('./dataEncoderDecoder_item_id.pkl', 'rb') as file:
            data = pickle.load(file)            
        
        digested_items = data['result']
        X_sparse = data['X_sparse']
        cv = data['cv']            
        
        # Convertir la matriz dispersa a una matriz densa si es necesario
        X = X_sparse.toarray()
        
        #with open('similarity_matrix.pkl', 'rb') as f:
        #    similarity = pickle.load(f)

        # Cargar la matriz dispersa desde el archivo, más eficiente para alamecenamiento ...
        #similarity_sparse_matrix = load_npz('./similarity_sparse.npz')
        
        # Convertir la matriz dispersa a densa si es necesario para el cálculo
        #similarity = similarity_sparse_matrix.toarray()            
        
        return digested_items, cv, X#, similarity_sparse_matrix 

    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None, None, None#, None
    
# Cargar los objetos desde los archivos pickle
digested_items, cv, X = cargar_pkls()
#similarity = cosine_similarity(X)

# Ahora puedes usar `result`, `cv` y `X` como lo hacías antes

if digested_items is None:
    # Cargar el fichero CSV
    df = pd.read_csv('./item_features_semirelleno_con_literales_actualizado.csv')
    # df = pd.read_csv('./item_features_encoderDecoder_v1.csv')  # Usar este si se prefiere
    # df = pd.read_csv('./item_features_encoderDecoder_v2.csv')  # Usar este si se prefiere

    # Eliminar filas con NaN en 'feature_category_literal' y 'feature_value_literal'
    df_complete = df.dropna(subset=['feature_category_literal', 'feature_value_literal'])

    # Definir los valores especiales para 'descrip'
    categorias_descriptivas = ['productos', 'materiales', 'caracteristicas', 'color', 'ocasion' , 'estaciones']

    # Crear una función para procesar cada grupo de item_id
    def procesar_grupo(grupo):
        grupo_copia = grupo.copy()
        # Filtrar los feature_value_literal que son considerados especiales y existen en el grupo
        valores_especiales_en_grupo = grupo_copia[grupo_copia['feature_category_literal'].isin(categorias_descriptivas)]

        # Ordenar 'valores_especiales_en_grupo' por el orden de 'categorias_descriptivas'
        valores_especiales_en_grupo['feature_category_literal'] = pd.Categorical(
            valores_especiales_en_grupo['feature_category_literal'], 
            categories=categorias_descriptivas,
            ordered=True
        )
        valores_especiales_en_grupo = valores_especiales_en_grupo.sort_values(by='feature_category_literal')

        # Concatenar los valores especiales para 'descrip' solo si existen en el item_id
        descrip = ' '.join(valores_especiales_en_grupo['feature_value_literal'])
        # Concatenar item_id al campo descrip con el formato '-(item_id)-'
        descrip += ' ({})'.format(grupo_copia['item_id'].iloc[0])


        # Crear la columna 'keywords' eliminando 'feature_category_literal' si está en 'feature_value_literal'
        keywords = ' '.join(
            grupo_copia['feature_value_literal'].apply(
                lambda x: x.replace(grupo_copia['feature_category_literal'].iloc[0], '') if grupo_copia['feature_category_literal'].iloc[0] in x else x
            ).unique()
        ).lower()

        return pd.Series({'item_descrip': descrip, 'item_keywords': keywords})

    # Aplicar la función al DataFrame agrupado por 'item_id'
    result = df_complete.groupby('item_id').apply(procesar_grupo).reset_index()

    # Mostrar el resultado
    print(result)

    def collapseDouble(L):
        if isinstance(L, list):
            return [i.replace("  ", " ") for i in L]
        return [] 

    #result.loc[:, 'item_keywords'] = result['item_keywords'].apply(collapseDouble)
    #result['item_keywords'] = result['item_keywords'].apply(lambda x:x.lower())
    # Convertir todos los valores de 'item_keywords' a string y luego a minúsculas
    #result['item_keywords'] = result['item_keywords'].astype(str).apply(lambda x: x.lower())


    print(result)
else:
    result = digested_items
    print("De datos datos cargados, X, cv, digested_items!")

Error: [Errno 2] No such file or directory: './dataEncoderDecoder_item_id.pkl'
       item_id                                     item_descrip  \
0            2       Camiseta Algodón Manga larga Primavera (2)   
1            3         Blusa Seda Manga corta Casual Verano (3)   
2            4        Sudadera Denim Largo Casual Primavera (4)   
3            7                   Vestido Cuero Corto Verano (7)   
4            8      Camiseta Seda Completo Casual Primavera (8)   
...        ...                                              ...   
23686    28139   Sandalias Cuero Completo Casual Verano (28139)   
23687    28140           Camiseta Piel sintética Verano (28140)   
23688    28141  Leggings Lino Amortiguada Cóctel Verano (28141)   
23689    28142           Botas Plástico Ajustable Otoño (28142)   
23690    28143            Camiseta Seda Completo Verano (28143)   

                                           item_keywords  
0      12-14 camiseta liso m primavera manga larga co... 

C:\Users\alcoholyvendas\AppData\Local\Temp\ipykernel_17132\3674956024.py:84: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df_complete.groupby('item_id').apply(procesar_grupo).reset_index()


In [21]:
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import coo_matrix

import numpy as np
from scipy.sparse import coo_matrix, save_npz
from sklearn.metrics.pairwise import cosine_similarity

def compute_and_save_sparse_similarity(X, threshold=0.1, filename='./similarity_sparse.npz', block_size=1000):
    """
    Calcula la matriz de similitud en bloques y guarda solo las similitudes que superan el umbral en formato disperso.
    
    Args:
    - X: La matriz de características (muestras x características).
    - threshold: El umbral de similitud para guardar valores.
    - filename: El nombre del archivo para guardar la matriz dispersa.
    - block_size: Tamaño del bloque para procesar las similitudes en partes.
    """
    n_samples = X.shape[0]
    data = []
    rows = []
    cols = []
    
    # Procesar la matriz en bloques para evitar problemas de memoria
    for start_row in range(0, n_samples, block_size):
        end_row = min(start_row + block_size, n_samples)
        # Extraer el bloque actual
        block = X[start_row:end_row]
        
        # Calcular similitudes para el bloque actual
        similarities = cosine_similarity(block, X)
        
        # Filtrar por el umbral y almacenar en formato disperso
        for i in range(end_row - start_row):
            for j in range(n_samples):
                sim = similarities[i, j]
                if sim > threshold:
                    data.append(sim)
                    rows.append(start_row + i)
                    cols.append(j)
    
    # Crear la matriz dispersa
    sparse_matrix = coo_matrix((data, (rows, cols)), shape=(n_samples, n_samples))
    
    # Guardar la matriz dispersa en un archivo
    save_npz(filename, sparse_matrix)
    print("Matriz dispersa de similaridad guardada!")

# Ejemplo de uso
# X = ... (tu matriz de características)
# compute_and_save_sparse_similarity(X)

    
if (cv is None or X is None):

    ps = PorterStemmer()
    
    def stem(text):
        y = []

        for i in text.split():
            y.append(ps.stem(i))

        return " ".join(y)

    result['item_keywords'] = result['item_keywords'].apply(stem)

    
    from nltk.corpus import stopwords
    # Descargar stopwords en español si no están ya disponibles
    nltk.download('stopwords')

    # Obtener la lista de stop words en español
    stop_words_spanish = list(stopwords.words('spanish'))

    # Crear CountVectorizer con la lista de stop words en español
    cv = CountVectorizer(max_features=5000, stop_words=stop_words_spanish)
    #cv = CountVectorizer(max_features=5000,stop_words='english')

    X = cv.fit_transform(result['item_keywords']).toarray()

    #print("get_feature_names_out",cv.get_feature_names_out())
    #print("get_stop_words",cv.get_stop_words())


    #calculate the cosine distance between each vector(movie)
    #cosine similarity is used because it expresses the similarity between the two objects
    
    #similarity = cosine_similarity(X)
    # Supongamos que `similarity` es una matriz densa
    #similarity_sparse = csr_matrix(similarity)

    # Guardar la matriz dispersa en un archivo
    #save_npz('./similarity_sparse.npz', csr_matrix(similarity))
    #compute_and_save_sparse_similarity(X)
    
    
    #print("similarity",similarity)
    #print("similarity[0]",similarity[0])
    print("Datos datos generados, count_vectorizer/features_matrix!")
else:
    print("De datos datos cargados, count_vectorizer/features_matrix!")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alcoholyvendas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Datos datos generados, count_vectorizer/features_matrix!


In [22]:
from datetime import datetime

def compute_query_similarity(query, X, cv, threshold=0.1):
    """
    Calcula la similitud entre la consulta y todos los ítems existentes usando una matriz dispersa guardada.
    
    Args:
    - query: La consulta para calcular similitudes.
    - X: La matriz de características (muestras x características).
    - cv: El CountVectorizer o TF-IDF Vectorizer utilizado para transformar la consulta.
    - sparse_matrix_filename: El nombre del archivo que contiene la matriz dispersa de similitud.
    - threshold: El umbral de similitud para considerar valores relevantes.
    
    Returns:
    - sparse_matrix: La matriz dispersa de similitudes.
    - query_similarity: La similitud de la consulta con todos los ítems.
    """
    ps = PorterStemmer()
    
    def stem(text):
        y = []

        for i in text.split():
            y.append(ps.stem(i))

        return " ".join(y)
    
    query_stemmed = stem(query.lower())
    print(f"query_stemmed:{query_stemmed}")
    
    # Transformar la consulta en un vector
    query_vector = cv.transform([query_stemmed]).toarray()
    print(f"query_vector:{query_vector}")
    
    # Obtener el vocabulario del CountVectorizer
    vocab = cv.vocabulary_

    # Invertir el diccionario de vocabulario para obtener un mapeo de índices a palabras
    index_to_word = {v: k for k, v in vocab.items()}

    # Mostrar las palabras correspondientes a los índices con valores distintos de cero
    for index, value in enumerate(query_vector[0]):
        if value > 0:
            print(f"Palabra: {index_to_word[index]}, Frecuencia: {value}")    
    
    
    
    # Cargar la matriz dispersa desde el archivo
    #sparse_matrix = load_npz(sparse_matrix_filename)
    
    
    # Calcular la similitud entre la consulta y todos los ítems
    query_similarity = cosine_similarity(query_vector, X).flatten()
    
    # Filtrar similitudes que superan el umbral
    data = []
    rows = []
    cols = []
    for i, sim in enumerate(query_similarity):
        if sim > threshold:
            data.append(sim)
            rows.append(0)  # Fila 0 para la consulta
            cols.append(i)
    
    # Crear la matriz dispersa de similitudes para la consulta
    query_sparse_matrix = coo_matrix((data, (rows, cols)), shape=(1, X.shape[0]))
    
    return query_sparse_matrix, query_similarity

def get_top_items(query_sparse_matrix, query_similarity, top_n=5):
    """
    Obtiene los índices de los ítems más similares desde la matriz dispersa.
    
    Args:
    - query_sparse_matrix: La matriz dispersa de similitudes para la consulta.
    - query_similarity: La similitud de la consulta con todos los ítems.
    - top_n: Número de ítems más similares a recuperar.
    
    Returns:
    - items_list: Lista de los ítems más similares.
    """
    # Convertir la matriz dispersa a densa si es necesario
    dense_similarity = query_sparse_matrix.toarray().flatten()
    
    # Obtener los índices de los ítems más similares
    item_indices = np.argsort(-dense_similarity)[:top_n]
    items_list = [(i, dense_similarity[i]) for i in item_indices]
    
    return items_list

# Esto genera un candidato a dar al recomendador, todas las preguntas podrían irse acumulando en la misma sesión
# para ir acumulando las queries e ir pasandolas todas cada vez que hay una nueva!
def encoding2recomender(yourQuery, session_id=1):

    query_sparse_matrix, query_similarity = compute_query_similarity(yourQuery, X, cv)
    items_list = get_top_items(query_sparse_matrix, query_similarity)    

    candidate_df = None
    # Crear la salida en formato CSV con solo el primer ítem
    if items_list:  # Verificar que hay elementos en items_list
        first_item_index = items_list[0][0]  # Obtener el índice del primer ítem
        item_id = result.iloc[first_item_index].item_id
        candidates = [{
            'session_id': session_id,
            'item_id': item_id,
            'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')  # Formatear la fecha
        }]
        
        # Convertir el resultado a un DataFrame
        candidate_df = pd.DataFrame(candidates)
    else:
        # Devolver un DataFrame vacío si items_list está vacío
        candidate_df = pd.DataFrame(columns=['session_id', 'item_id', 'date'])        
    
    return candidate_df        
        
        
# Ejemplo de uso
encoding2recomender("Ahora que llega el verano, me gustaría una camisa blanca para más veranos, quizá de algodón o que me recomiendas")


query_stemmed:ahora que llega el verano, me gustaría una camisa blanca para má veranos, quizá de algodón o que me recomienda
query_vector:[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 

,session_id,item_id,date
0,1,189,2024-08-31 18:10:34.602896


In [23]:
import pickle
from scipy.sparse import csr_matrix, save_npz

X_sparse = csr_matrix(X)
data = {
    'result': result,
    'X_sparse': X_sparse,
    'cv': cv
}
# Guardar el diccionario en un archivo pickle
with open('./dataEncoderDecoder_item_id.pkl', 'wb') as file:
    pickle.dump(data, file)

# Esto es lo que hay que cargar en la api function, además del modelo ...
# el enconding2recomender
#pickle.dump(result, open('./digested_items.pkl', 'wb'))
#pickle.dump(cv, open('./count_vectorizer.pkl', 'wb'))
#pickle.dump(X, open('./features_matrix.pkl', 'wb'))

# Apartir de aquí, creo que no haría falta cargarlo! (la similaridad se hace por el par X -matriz dispersa- y la query)
#pickle.dump(result.to_dict(), open('./digested_items_dict.pkl', 'wb'))
#pickle.dump(similarity, open('./similarity_matrix.pkl', 'wb'))
#np.save('similarity_matrix.npy', similarity)  # Guarda en disco, si sólo hay que guardar eso!
#Pero si hay problemas de tamaño, lo siguiente es lo mejor!

In [24]:
# Fase de decodificación!
# Si esto es la salida del recomendador, hay de decodearlo para devolverlo al usuario (respuesta del chat)
# Siempre será una sesión nada más, pero para ver variedad!
prediccion_df = pd.DataFrame({
    'session_id': [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61],
    'item_id': [3260, 5383, 26538, 20541, 2213, 6603, 27921, 3425, 7368, 7719, 25956, 13936, 16877, 7225, 11641, 14075, 20236, 18298, 10207, 14455],
    'rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
})

def decoding2chat(prediccion_df):
    """
    Añade la columna 'item_descrip' a prediccion_df basándose en el DataFrame result.
    
    Parameters:
    prediccion_df (pd.DataFrame): DataFrame con columnas 'session_id', 'item_id', 'rank'.
    result (pd.DataFrame): DataFrame con las columnas 'item_id', 'item_descrip', 'item_keywords'.
    
    Returns:
    pd.DataFrame: prediccion_df con la nueva columna 'item_descrip'.
    """

    # Crear un diccionario para la búsqueda rápida
    item_descrip_dict = result.set_index('item_id')['item_descrip'].to_dict()

    # Función para obtener la descripción del item
    def obtener_descripcion(item_id):
        return item_descrip_dict.get(item_id, '')

    # Aplicar la función a cada item_id en prediccion_df
    prediccion_df['item_descrip'] = prediccion_df['item_id'].apply(obtener_descripcion)

    #return prediccion_df
    #joined_df = decoded_prediccion_df.groupby('session_id')['item_descrip'].apply(lambda x: ', '.join(x)).reset_index()
    listed_df = prediccion_df.groupby('session_id')['item_descrip'].apply(list).reset_index()
    
    return listed_df

# Mostrar el DataFrame final con la columna añadida
decoded_prediccion_df = decoding2chat(prediccion_df)
#print(decoded_prediccion_df)

import json

# Mostrar el DataFrame de 'decoded_prediccion_df'
# Acceder a la primera fila de listed_df al valor 'item_descrip'
#lista = decoded_prediccion_df.iloc[0]
#print(lista)
print(json.dumps(decoded_prediccion_df.iloc[0]['item_descrip'], ensure_ascii=False))

# Impresión si quieres, iterar por cada fila en 'listed_df' y convertirla a JSON
for index, row in decoded_prediccion_df.iterrows():
    session_id = row['session_id']
    item_descriptions = row['item_descrip']
    
    # Crear un diccionario para representar la fila
    data = {
        'session_id': session_id,
        'item_descrip': item_descriptions
    }
    
    # Convertir el diccionario a JSON y imprimirlo
    json_data = json.dumps(item_descriptions, ensure_ascii=False)
    print(session_id, json_data)


#listed_df


["Cazadora Seda Completo Casual Primavera (3260)", "Americana Seda Completo Casual Verano (5383)", "Camiseta Seda Amortiguada Casual Primavera (26538)", "Leggings Seda Completo Casual Verano (20541)", "Cazadora Seda Completo Casual Verano (2213)", "Camiseta Seda Corto Casual Verano (6603)", "Cazadora Seda Corto Casual Verano (27921)", "Camiseta Nylon Elástico Casual Verano (3425)", "Cazadora Seda Corto Casual Verano (7368)", "Camiseta Cuero Completo Casual Primavera (7719)"]
26 ["Cazadora Seda Completo Casual Primavera (3260)", "Americana Seda Completo Casual Verano (5383)", "Camiseta Seda Amortiguada Casual Primavera (26538)", "Leggings Seda Completo Casual Verano (20541)", "Cazadora Seda Completo Casual Verano (2213)", "Camiseta Seda Corto Casual Verano (6603)", "Cazadora Seda Corto Casual Verano (27921)", "Camiseta Nylon Elástico Casual Verano (3425)", "Cazadora Seda Corto Casual Verano (7368)", "Camiseta Cuero Completo Casual Primavera (7719)"]
61 ["Zapatillas de casa Cuero Complet